# Train a Pytorch model with SparkXshards

Copyright 2016 The BigDL Authors.

SparkXshards in Orca allows users to process large-scale dataset using existing Python codes in a distributed and data-parallel fashion, as shown below. This notebook is an example of how to train a pytorch model using data of SparkXshards on Orca. 

It is adapted from [PyTorch Tutorial: How to Develop Deep Learning Models with Python](https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/)

In [1]:
# import necessary libraries
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

import bigdl.orca.data.pandas
from bigdl.orca import init_orca_context, stop_orca_context
from bigdl.orca.learn.pytorch import Estimator
from bigdl.orca.learn.metrics import Accuracy
from bigdl.orca.data.transformer import StringIndexer
import os

os.environ['KMP_DUPLICATE_LIB_OK'] ='True'


/Users/guoqiong/opt/anaconda3/envs/py37tf2_x/lib/python3.7/site-packages/bigdl/dllib/utils/engine.py:47: UserWarning: Find both SPARK_HOME and pyspark. You may need to check whether they match with each other. SPARK_HOME environment variable is set to: /Users/guoqiong/intelWork/tools/spark/spark-2.4.6-bin-hadoop2.7, and pyspark is found in: /Users/guoqiong/opt/anaconda3/envs/py37tf2_x/lib/python3.7/site-packages/pyspark/__init__.py. If they are unmatched, please use one source only to avoid conflict. For example, you can unset SPARK_HOME and use pyspark only.
  warnings.warn(warning_msg)
/Users/guoqiong/opt/anaconda3/envs/py37tf2_x/lib/python3.7/site-packages/bigdl/dllib/utils/zoo_engine.py:47: UserWarning: Find both SPARK_HOME and pyspark. You may need to check whether they match with each other. SPARK_HOME environment variable is set to: /Users/guoqiong/intelWork/tools/spark/spark-2.4.6-bin-hadoop2.7, and pyspark is found in: /Users/guoqiong/opt/anaconda3/envs/py37tf2_x/lib/python3.7

In [2]:
# start an OrcaContext
sc = init_orca_context(cores=4, memory="4g")

Initializing orca context
Current pyspark location is : /Users/guoqiong/opt/anaconda3/envs/py37tf2_x/lib/python3.7/site-packages/pyspark/__init__.py
Start to getOrCreate SparkContext
pyspark_submit_args is:  --driver-class-path /Users/guoqiong/opt/anaconda3/envs/py37tf2_x/lib/python3.7/site-packages/bigdl/share/core/lib/all-2.2.0-20220919.010507-1.jar:/Users/guoqiong/opt/anaconda3/envs/py37tf2_x/lib/python3.7/site-packages/bigdl/share/dllib/lib/bigdl-dllib-spark_2.4.6-2.2.0-SNAPSHOT-jar-with-dependencies.jar:/Users/guoqiong/opt/anaconda3/envs/py37tf2_x/lib/python3.7/site-packages/bigdl/share/orca/lib/bigdl-orca-spark_2.4.6-2.2.0-SNAPSHOT-jar-with-dependencies.jar pyspark-shell 
2022-11-23 23:48:53 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-11-23 23:48:53 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2022-11-23 23:48:53 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2022-11-23 23:48:54,889 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-11-23 23:48:54,892 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-11-23 23:48:54,893 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-11-23 23:48:54,893 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
22-11-23 23:48:54 [Thread-3] INFO  Engine$:122 - Auto detect executor number and executor cores number
22-11-23 23:48:54 [Thread-3] INFO  Engine$:124 - Executor number is 1 and executor cores number is 4



User settings:

   KMP_AFFINITY=granularity=fine,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_SETTINGS=1
   OMP_NUM_THREADS=1

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=0
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_INIT_WAIT=2048
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NEXT_WAIT=1024
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
 

22-11-23 23:48:55 [Thread-3] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 14
2022-11-23 23:48:55 WARN  SparkContext:66 - Using an existing SparkContext; some configuration may not take effect.
22-11-23 23:48:55 [Thread-3] INFO  Engine$:461 - Find existing spark context. Checking the spark conf...
cls.getname: com.intel.analytics.bigdl.dllib.utils.python.api.Sample
BigDLBasePickler registering: bigdl.dllib.utils.common  Sample
cls.getname: com.intel.analytics.bigdl.dllib.utils.python.api.EvaluatedResult
BigDLBasePickler registering: bigdl.dllib.utils.common  EvaluatedResult
cls.getname: com.intel.analytics.bigdl.dllib.utils.python.api.JTensor
BigDLBasePickler registering: bigdl.dllib.utils.common  JTensor
cls.getname: com.intel.analytics.bigdl.dllib.utils.python.api.JActivity
BigDLBasePickler registering: bigdl.dllib.utils.common  JActivity
Successfully got a SparkContext


## Load data in parallel and get general information

Load data into data_shards, it is a SparkXshards that can be operated on in parallel, here each element of the data_shards is a panda dataframe read from a file on the cluster. Users can distribute local code of `pd.read_csv(dataFile)` using `bigdl.orca.data.pandas.read_csv(datapath)`.

In [3]:
data_shards = bigdl.orca.data.pandas.read_csv('/Users/guoqiong/intelWork/data/ionosphere/ionosphere.csv', header=None)

2022-11-23 23:48:55,862 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-11-23 23:48:55,863 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-11-23 23:48:55,864 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-11-23 23:48:55,865 Thread-3 WARN The bufferSize is set to 4000 but bufferedIo is false: false
22-11-23 23:48:55 [Thread-3] INFO  Engine$:122 - Auto detect executor number and executor cores number
22-11-23 23:48:55 [Thread-3] INFO  Engine$:124 - Executor number is 1 and executor cores number is 4
22-11-23 23:48:55 [Thread-3] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 14
2022-11-23 23:48:55 WARN  SparkContext:66 - Using an existing SparkContext; some configuration may not take effect.
22-11-23 23:48:55 [Thread-3] INFO  Engine$:461 - Find existing spark context. Checking the spark conf...
create shards from Spark DataFrame attempted Arrow optimization failed as: name 'df' is not d

In [4]:
# show the first couple of rows in the data_shards
data_shards.head(5)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300,g
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447,b
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238,g
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000,b
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697,g


In [5]:
# see the num of partitions of data_shards
data_shards.num_partitions()


1

In [6]:
# count total number of rows in the data_shards
len(data_shards)

351

In [7]:
# columns information of element of data_shards.
columns = data_shards.get_schema()['columns']

##  Encode labels

The labels are in strings. Users can transform the strings into integers using `StringIndexer`

In [8]:
label_encoder = StringIndexer(inputCol=columns[-1])
data_shards = label_encoder.fit_transform(data_shards)

createDataFrame from shards attempted Arrow optimization failed as: 'NoneType' object has no attribute 'json',Will try without Arrow optimization
2022-11-23 23:49:03 WARN  Utils:66 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


create shards from Spark DataFrame attempted Arrow optimization failed as: name 'df' is not defined. Will try without Arrow optimization


Labels start from 1 so need to be updated to zero based.

In [9]:
def update_label_to_zero_base(df):
    df['34'] = df['34'] - 1
    return df
data_shards = data_shards.transform_shard(update_label_to_zero_base)

## Assemble feature and labels

In [10]:
data_shards = data_shards.assembleFeatureLabelCols(featureCols=list(columns[:-1]),
                                                   labelCols=[columns[-1]])

## Define PyTorch model and train it

Users can build a PyTorch model as usual and use Orca Estimator to train it

In [11]:
# define a MLP model
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()

    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X

model = MLP(34)

In [12]:
# define criterion and optimizer
criterion = BCELoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

In [13]:
# build Orca Estimator
orca_estimator = Estimator.from_torch(model=model,
                                      optimizer=optimizer,
                                      loss=criterion,
                                      metrics=[Accuracy()],
                                      backend="bigdl")


creating: createTorchLoss
creating: createTorchOptim
creating: createZooKerasAccuracy
creating: createEstimator


In [14]:
# train the model
orca_estimator.fit(data=data_shards, epochs=100, batch_size=32)

creating: createMaxEpoch
22-11-23 23:49:07 [Thread-3] INFO  InternalDistriOptimizer$:1009 - TorchModel[840e212d] isTorch is true
22-11-23 23:49:07 [Thread-3] INFO  InternalDistriOptimizer$:1016 - torch model will use 1 OMP threads.
22-11-23 23:49:07 [Thread-3] INFO  DistriOptimizer$:830 - caching training rdd ...
22-11-23 23:49:08 [Thread-3] INFO  DistriOptimizer$:655 - Cache thread models...
22-11-23 23:49:08 [Executor task launch worker for task 615] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 55
22-11-23 23:49:08 [Executor task launch worker for task 615] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 55
22-11-23 23:49:08 [Executor task launch worker for task 615] INFO  DistriOptimizer$:638 - model thread pool size is 1
2022-11-23 23:49:08 WARN  BlockManager:66 - Asked to remove block test_0weights0, which does not exist
2022-11-23 23:49:08 WARN  BlockManager:66 - Asked to remove block test_0gradients0, which does not exist
22-11-23 23:49:08 [Thread-3] INFO  DistriO

[Stage 26:>                                                         (0 + 1) / 1]
User settings:

   KMP_AFFINITY=granularity=fine,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_SETTINGS=1
   OMP_NUM_THREADS=1

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_MONOTONIC_DYNAMIC_SCHEDULE=false
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=0
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=th

22-11-23 23:49:10 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 1 32/351][Iteration 1][Wall Clock 1.740796068s] Trained 32.0 records in 1.740796068 seconds. Throughput is 18.382395 records/second. Loss is 0.83389026. 
22-11-23 23:49:10 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 1 64/351][Iteration 2][Wall Clock 1.796852284s] Trained 32.0 records in 0.056056216 seconds. Throughput is 570.8555 records/second. Loss is 0.9555579. 
22-11-23 23:49:10 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 1 96/351][Iteration 3][Wall Clock 1.848830414s] Trained 32.0 records in 0.05197813 seconds. Throughput is 615.64355 records/second. Loss is 0.8175985. 
22-11-23 23:49:10 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 1 128/351][Iteration 4][Wall Clock 1.905651245s] Trained 32.0 records in 0.056820831 seconds. Throughput is 563.1737 records/second. Loss is 0.80436933. 
22-11-23 23:49:10 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 1 160/351][Iteration 5][Wall Clock 1.96153022s] Trained 32.0 re

22-11-23 23:49:12 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 4 256/351][Iteration 41][Wall Clock 3.788529421s] Trained 32.0 records in 0.03794508 seconds. Throughput is 843.3241 records/second. Loss is 0.39056805. 
22-11-23 23:49:12 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 4 288/351][Iteration 42][Wall Clock 3.825663594s] Trained 32.0 records in 0.037134173 seconds. Throughput is 861.7399 records/second. Loss is 0.4246154. 
22-11-23 23:49:12 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 4 320/351][Iteration 43][Wall Clock 3.860311601s] Trained 32.0 records in 0.034648007 seconds. Throughput is 923.574 records/second. Loss is 0.42755866. 
22-11-23 23:49:12 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 4 352/351][Iteration 44][Wall Clock 3.896763343s] Trained 32.0 records in 0.036451742 seconds. Throughput is 877.873 records/second. Loss is 0.47986147. 
22-11-23 23:49:12 [Thread-3] INFO  DistriOptimizer$:474 - [Epoch 4 352/351][Iteration 44][Wall Clock 3.896763343s] Epoch fin

22-11-23 23:49:13 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 8 32/351][Iteration 78][Wall Clock 5.394895181s] Trained 32.0 records in 0.029445669 seconds. Throughput is 1086.7473 records/second. Loss is 0.36214057. 
22-11-23 23:49:14 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 8 64/351][Iteration 79][Wall Clock 5.438350305s] Trained 32.0 records in 0.043455124 seconds. Throughput is 736.39185 records/second. Loss is 0.36939612. 
22-11-23 23:49:14 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 8 96/351][Iteration 80][Wall Clock 5.468070122s] Trained 32.0 records in 0.029719817 seconds. Throughput is 1076.7227 records/second. Loss is 0.39776292. 
22-11-23 23:49:14 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 8 128/351][Iteration 81][Wall Clock 5.497318991s] Trained 32.0 records in 0.029248869 seconds. Throughput is 1094.0594 records/second. Loss is 0.354247. 
22-11-23 23:49:14 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 8 160/351][Iteration 82][Wall Clock 5.535657653s] Traine

...

22-11-23 23:49:33 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 86 352/351][Iteration 946][Wall Clock 24.536876432s] Trained 32.0 records in 0.050343651 seconds. Throughput is 635.6313 records/second. Loss is 0.0060224365. 
22-11-23 23:49:33 [Thread-3] INFO  DistriOptimizer$:474 - [Epoch 86 352/351][Iteration 946][Wall Clock 24.536876432s] Epoch finished. Wall clock time is 24542.541125 ms
22-11-23 23:49:33 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 87 32/351][Iteration 947][Wall Clock 24.559136564s] Trained 32.0 records in 0.016595439 seconds. Throughput is 1928.2407 records/second. Loss is 0.022977024. 
22-11-23 23:49:33 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 87 64/351][Iteration 948][Wall Clock 24.57506061s] Trained 32.0 records in 0.015924046 seconds. Throughput is 2009.5396 records/second. Loss is 0.022050738. 
22-11-23 23:49:33 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 87 96/351][Iteration 949][Wall Clock 24.60306225s] Trained 32.0 records in 0.02800164 seconds.

22-11-23 23:49:33 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 90 160/351][Iteration 984][Wall Clock 25.17417356s] Trained 32.0 records in 0.016129204 seconds. Throughput is 1983.979 records/second. Loss is 0.024189793. 
22-11-23 23:49:33 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 90 192/351][Iteration 985][Wall Clock 25.18974718s] Trained 32.0 records in 0.01557362 seconds. Throughput is 2054.7568 records/second. Loss is 0.013346303. 
22-11-23 23:49:33 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 90 224/351][Iteration 986][Wall Clock 25.208178304s] Trained 32.0 records in 0.018431124 seconds. Throughput is 1736.1937 records/second. Loss is 0.029657107. 
22-11-23 23:49:33 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 90 256/351][Iteration 987][Wall Clock 25.225159068s] Trained 32.0 records in 0.016980764 seconds. Throughput is 1884.4854 records/second. Loss is 0.014375692. 
22-11-23 23:49:33 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 90 288/351][Iteration 988][Wall Clock 2

22-11-23 23:49:34 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 93 288/351][Iteration 1021][Wall Clock 25.80989039s] Trained 32.0 records in 0.018178833 seconds. Throughput is 1760.289 records/second. Loss is 0.023569778. 
22-11-23 23:49:34 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 93 320/351][Iteration 1022][Wall Clock 25.824189725s] Trained 32.0 records in 0.014299335 seconds. Throughput is 2237.8662 records/second. Loss is 0.021187522. 
22-11-23 23:49:34 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 93 352/351][Iteration 1023][Wall Clock 25.83964463s] Trained 32.0 records in 0.015454905 seconds. Throughput is 2070.54 records/second. Loss is 0.015953751. 
22-11-23 23:49:34 [Thread-3] INFO  DistriOptimizer$:474 - [Epoch 93 352/351][Iteration 1023][Wall Clock 25.83964463s] Epoch finished. Wall clock time is 25845.711002 ms
22-11-23 23:49:34 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 94 32/351][Iteration 1024][Wall Clock 25.861089149s] Trained 32.0 records in 0.015378147 seco

22-11-23 23:49:35 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 97 96/351][Iteration 1059][Wall Clock 26.430794289s] Trained 32.0 records in 0.016006038 seconds. Throughput is 1999.2456 records/second. Loss is 0.012013636. 
22-11-23 23:49:35 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 97 128/351][Iteration 1060][Wall Clock 26.446652067s] Trained 32.0 records in 0.015857778 seconds. Throughput is 2017.9371 records/second. Loss is 0.004162508. 
22-11-23 23:49:35 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 97 160/351][Iteration 1061][Wall Clock 26.463304026s] Trained 32.0 records in 0.016651959 seconds. Throughput is 1921.6959 records/second. Loss is 0.024014393. 
22-11-23 23:49:35 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 97 192/351][Iteration 1062][Wall Clock 26.478173235s] Trained 32.0 records in 0.014869209 seconds. Throughput is 2152.0984 records/second. Loss is 0.006278361. 
22-11-23 23:49:35 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 97 224/351][Iteration 1063][Wall

22-11-23 23:49:35 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 100 256/351][Iteration 1097][Wall Clock 27.061248236s] Trained 32.0 records in 0.01990746 seconds. Throughput is 1607.4376 records/second. Loss is 0.1620147. 
22-11-23 23:49:35 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 100 288/351][Iteration 1098][Wall Clock 27.079087234s] Trained 32.0 records in 0.017838998 seconds. Throughput is 1793.8228 records/second. Loss is 0.021568436. 
22-11-23 23:49:35 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 100 320/351][Iteration 1099][Wall Clock 27.095604293s] Trained 32.0 records in 0.016517059 seconds. Throughput is 1937.3909 records/second. Loss is 0.007176501. 
22-11-23 23:49:35 [Thread-3] INFO  DistriOptimizer$:429 - [Epoch 100 352/351][Iteration 1100][Wall Clock 27.114401855s] Trained 32.0 records in 0.018797562 seconds. Throughput is 1702.3485 records/second. Loss is 0.019158375. 
22-11-23 23:49:35 [Thread-3] INFO  DistriOptimizer$:474 - [Epoch 100 352/351][Iteration 1100][W

In [15]:
# stop OrcaContext
stop_orca_context()

Stopping orca context
